# Data Science Salary Prediction

## Column Description
1. `job_title`:	The job title or role associated with the reported salary.
2. `experience_level`:	The level of experience of the individual.
3. `employment_type`:	Indicates whether the employment is full-time, part-time, etc.
4. `work_models`:	Describes different working models (remote, on-site, hybrid).
5. `work_year`:	The specific year in which the salary information was recorded.
6. `employee_residence`:	The residence location of the employee.
7. `salary`:	The reported salary in the original currency.
8. `salary_currency`:	The currency in which the salary is denominated.
9. `salary_in_usd`:	The converted salary in US dollars.
10. `company_location`:	The geographic location of the employing organization.
11. `company_size`:	The size of the company, categorized by the number of employees.


Data source: https://www.kaggle.com/datasets/sazidthe1/data-science-salaries

In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

salary_df = pd.read_csv('./dataset/data_science_salaries.csv')
salary_df.head()

,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
0,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
2,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
4,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium


In [2]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11087 entries, 0 to 11086
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job_title           11087 non-null  object
 1   experience_level    11087 non-null  object
 2   employment_type     11087 non-null  object
 3   work_models         11087 non-null  object
 4   work_year           11087 non-null  int64 
 5   employee_residence  11087 non-null  object
 6   salary              11087 non-null  int64 
 7   salary_currency     11087 non-null  object
 8   salary_in_usd       11087 non-null  int64 
 9   company_location    11087 non-null  object
 10  company_size        11087 non-null  object
dtypes: int64(3), object(8)
memory usage: 952.9+ KB


In [3]:
salary_df.describe()

,work_year,salary,salary_in_usd
count,11087.000000,1.108700e+04,11087.000000
mean,2022.848381,1.695723e+05,149614.977631
std,0.567803,4.080311e+05,66704.329347
min,2020.000000,1.400000e+04,15000.000000
25%,2023.000000,1.050000e+05,104000.000000
50%,2023.000000,1.422000e+05,142000.000000
75%,2023.000000,1.880500e+05,185900.000000
max,2024.000000,3.040000e+07,750000.000000


In [4]:
salary_df.shape

(11087, 11)

In [5]:
# check unique values of categorical columns
columns = salary_df.columns.to_list()
numerical_columns = ['work_year', 'salary', 'salary_in_usd']
categorical_columns = list(set(columns).difference(numerical_columns))

categorical = salary_df[categorical_columns]
for cat in categorical_columns:
    print(f'Unique Values for {cat.upper()}: \n{salary_df[cat].unique()} \
          \nTotal Unique Values: {len(salary_df[cat].unique())}\n\n')

Unique Values for SALARY_CURRENCY: 
['USD' 'GBP' 'EUR' 'TRY' 'THB' 'ZAR' 'SGD' 'INR' 'PLN' 'PHP' 'NOK' 'ILS'
 'HKD' 'AUD' 'CHF' 'CAD' 'BRL' 'JPY' 'HUF' 'DKK' 'CLP' 'MXN']           
Total Unique Values: 22


Unique Values for EMPLOYEE_RESIDENCE: 
['United States' 'Romania' 'Portugal' 'Lebanon' 'India' 'Ireland'
 'United Kingdom' 'Spain' 'Egypt' 'Germany' 'Canada' 'Australia' 'Vietnam'
 'Uganda' 'Italy' 'Thailand' 'Philippines' 'Tunisia' 'Belgium' 'Turkey'
 'Nigeria' 'Uzbekistan' 'Argentina' 'France' 'Kuwait' 'Ukraine'
 'South Africa' 'Slovenia' 'Singapore' 'Sweden' 'Saudi Arabia' 'Armenia'
 'Russia' 'Georgia' 'Qatar' 'Peru' 'Poland' 'New Zealand' 'Norway'
 'Netherlands' 'Mexico' 'Latvia' 'Pakistan' 'Lithuania' 'South Korea'
 'Kenya' 'Moldova' 'Israel' 'Hungary' 'Croatia' 'Hong Kong' 'Greece'
 'Ghana' 'Finland' 'Estonia' 'Cyprus' 'Ecuador' 'Austria' 'Colombia'
 'Switzerland' 'Central African Republic' 'Czech Republic' 'Brazil'
 'Bosnia and Herzegovina' 'Andorra' 'China' 'Costa Rica' 'Ch

In [6]:
# convert unique categorical values < 10 to category dtype to save memory usage
cat_columns_small = [cat for cat in categorical_columns if len(salary_df[cat].unique()) < 10]
cat_columns_small

['employment_type', 'company_size', 'work_models', 'experience_level']

In [7]:
for cat in cat_columns_small:
    salary_df[cat] = salary_df[cat].astype('category')
    
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11087 entries, 0 to 11086
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   job_title           11087 non-null  object  
 1   experience_level    11087 non-null  category
 2   employment_type     11087 non-null  category
 3   work_models         11087 non-null  category
 4   work_year           11087 non-null  int64   
 5   employee_residence  11087 non-null  object  
 6   salary              11087 non-null  int64   
 7   salary_currency     11087 non-null  object  
 8   salary_in_usd       11087 non-null  int64   
 9   company_location    11087 non-null  object  
 10  company_size        11087 non-null  category
dtypes: category(4), int64(3), object(4)
memory usage: 650.4+ KB


In [8]:
# clean job title
# philosophy -> e.g. for DA: no matter if they are a lead, manager, their core title is still data analyst

def job_filter(string):
    return salary_df.job_title.str.contains(string)

ds_filter = job_filter('D.* Scien.*')
de_filter = job_filter('Da.* Engineer.*')
ml_filter = job_filter('M.* Engineer.*')

# DA Cleaning
da_filter = (job_filter('Analy')) & (~job_filter('Engine.*')) & (~job_filter('Manager')) \
                | job_filter('Da.* Vi.* Sp.*') | job_filter('Bu.* In.* Sp.*')
salary_df.loc[da_filter, 'job_title'] = 'Data Analyst'

In [9]:
# DE Cleaning
de_filter = (job_filter('Da.* Engineer.*')) & (~job_filter('Manager')) \
                | job_filter('Bu.* In.* En.*') | job_filter('BI De.*') \
                | job_filter('Bu.* In.* Dev.*') | job_filter('ETL.* En.*') \
                | job_filter('Ana.* En.*') | job_filter('Inte.*')
salary_df.loc[de_filter, 'job_title'] = 'Data Engineer'

In [10]:
# MLE Cleaning
mle_filter = (job_filter('Machine.* Engi.*')) & (~job_filter('Manager')) \
                | job_filter('ML.* E.*') | job_filter('NLP E.*') \
                | job_filter('Mach.* Dev.*') | (job_filter('Dee.* Lear.* En.*')) \
                | job_filter('Co.* Vis.* En*') | job_filter('Mac.* Le.* Mod.*') \
                | job_filter('AI Dev.*') | job_filter('AI Arch.*') | job_filter('AI Pro.*') \
                | job_filter('AI Eng.*')
salary_df.loc[mle_filter, 'job_title'] = 'Machine Learning Engineer'

In [11]:
# MLS Cleaning
mls_filter = (job_filter('Mac.* Sci.*')) & (~job_filter('Manager')) \
                | job_filter('Mac.* Le.* Sp.*') | job_filter('Mac.* Lea.* Re.*') \
                | job_filter('De.* Le.* Re.*')
salary_df.loc[mls_filter, 'job_title'] = 'Machine Learning Scientist'

In [12]:
# DS Cleaning
ds_filter = (job_filter('Da.* Sci.*')) & (~job_filter('Manager')) \
             & (~job_filter('Director')) & (~job_filter('Head')) \
             | (job_filter('Ap.* Sc.*')) | (job_filter('De.* Sci.*')) \
             | job_filter('AI.* Sc.*') | job_filter('AI.* E.*')
salary_df.loc[ds_filter, 'job_title'] = 'Data Scientist'

# DS Director Cleaning
ds_dir_filter = (job_filter('Da.* Sci.*')) & (job_filter('Director'))
salary_df.loc[ds_dir_filter, 'job_title'] = 'Data Science Director'

# Head of DS Cleaning
ds_head_filter = (job_filter('Da.* Sci.*') & job_filter('Head'))
salary_df.loc[ds_dir_filter, 'job_title'] = 'Head of Data Science'

# Head of Data Cleaning
hod_filter = job_filter('Head.* Data')
salary_df.loc[hod_filter, 'job_title'] = 'Head of Data Science'

# Data Manager Cleaning
data_man_filter = (job_filter('Data.* Man.*'))
salary_df.loc[data_man_filter, 'job_title'] = 'Data Manager'

In [13]:
# Data Architect Cleaning
darch_filter = (job_filter('Da.* Arc.*')) & (~job_filter('Manager')) \
             & (~job_filter('Director')) & (~job_filter('Head')) \
             | (job_filter('ETL.* De.*')) | job_filter('Da.* Dev.*') \
             | (job_filter('Da.* Mod.*'))
salary_df.loc[darch_filter, 'job_title'] = 'Data Architect'

In [14]:
salary_df.job_title.value_counts()

Data Engineer                    3192
Data Scientist                   2854
Data Analyst                     1977
Machine Learning Engineer        1536
Research Scientist                345
Data Architect                    344
Data Manager                      317
Research Engineer                 199
Machine Learning Scientist        122
Head of Data Science               86
Data Specialist                    56
Data Strategist                    20
Data Lead                          18
Prompt Engineer                     4
Machine Learning Manager            4
Data Operations Specialist          4
Data Product Owner                  4
Head of Machine Learning            3
Autonomous Vehicle Technician       2
Name: job_title, dtype: int64

In [15]:
# based on the numbers above, can put ml head -> ds head
# ml manager -> ds manager
# sum job titles <= 20 are to be dropped because it's really imbalanced